# Algorithm performance and running time comparison on synthetic data (Figures EC.2, EC.6, EC.7)

In this notebook we produce plots to compare the performance of the algorithms presented in the paper in terms of utility for instances with general costs. The resulting plots are presented in Figures EC.2, EC.6 & EC.7 of the Electronic Companion.

**Required script execution:** khayyam.sh

In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
from lib import utils
sns.set_context("paper", font_scale=4)

Here, set the value of $\kappa$ (choose from \{$0.25$, $0.5$, $0.75$\}) and the type of cost function (choose from \{uniform, exponential\}).

In [3]:
name="khayyam"
cost_method="uniform"
kappa=0.25
final_df = []
cols = sns.color_palette("Set2",9)
col_dict = {"Non-Strategic": cols[0],
            "Bruteforce": cols[1],
            "Threshold": cols[2],
            "Iterative": cols[3],
            "DP": cols[5]
           }
linestyles_dict = {"Non-Strategic": "-.",
            "Bruteforce": "-",
            "Threshold": "-",
            "Iterative": "--",
            "DP": ":"
           }
markers_dict = {"Non-Strategic": "v",
            "Bruteforce": "s",
            "Threshold": "^",
            "Iterative": "o",
            "DP": "D"
           }

In [4]:
files = glob.glob("./outputs/optimal/{name}_bruteforce_cost_{cost_method}_*_kappa_{kappa}_*_config.json".format(name=name, cost_method=cost_method, kappa=kappa))
data = []

for fi in files:
    with open(fi,"r") as file:
        data.append(json.load(file))
df_standard = pd.DataFrame(data)
final_df.append(df_standard.melt(id_vars=["m", "seed", "kappa", "sparsity", "time", "pi", "bruteforce_br"], var_name="algorithm", value_name="utility"))

In [5]:
files = glob.glob("./outputs/optimal/{name}_thres_cost_{cost_method}_*_kappa_{kappa}_*_config.json".format(name=name, cost_method=cost_method, kappa=kappa))
data = []

for fi in files:
    with open(fi,"r") as file:
        data.append(json.load(file))
df_standard = pd.DataFrame(data)
final_df.append(df_standard.melt(id_vars=["m", "seed", "kappa", "sparsity", "time", "pi", "strategic_threshold_br"], var_name="algorithm", value_name="utility"))

In [6]:
files = glob.glob("./outputs/optimal/{name}_iterative_cost_{cost_method}_*_kappa_{kappa}_*_config.json".format(name=name, cost_method=cost_method, kappa=kappa))
data = []

for fi in files:
    with open(fi,"r") as file:
        data.append(json.load(file))
df_standard = pd.DataFrame(data)
final_df.append(df_standard.melt(id_vars=["m", "seed", "kappa", "sparsity", "time", "parallel", "split_components", "iterations", "pi", "strategic_br", "pi_non_strategic", "non_strategic_br", "strategic_deter_br", "pi_strategic_deter", "strategic_deter"], var_name="algorithm", value_name="utility"))

In [7]:
df = pd.concat(final_df)
df['algorithm'][df['algorithm']=="strategic"] = df['algorithm'][df['algorithm']=="strategic"]+df['parallel'][df['algorithm']=="strategic"].astype(str)

/var/folders/gy/xwt8ngc15hj3w_8vq_bk0yz80000gt/T/ipykernel_93946/2958989433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['algorithm'][df['algorithm']=="strategic"] = df['algorithm'][df['algorithm']=="strategic"]+df['parallel'][df['algorithm']=="strategic"].astype(str)
/var/folders/gy/xwt8ngc15hj3w_8vq_bk0yz80000gt/T/ipykernel_93946/2958989433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['algorithm'][df['algorithm']=="strategic"] = df['algorithm'][df['algorithm']=="strategic"]+df['parallel'][df['algorithm']=="strategic"].astype(str)


In [8]:
df['algorithm'].unique()

array(['bruteforce', 'strategic_threshold', 'strategicFalse',
       'non_strategic'], dtype=object)

In [9]:
df['algorithm'] = df['algorithm'].str.replace("bruteforce", "Bruteforce")
df['algorithm'] = df['algorithm'].str.replace("non_strategic", "Non-Strategic")
df['algorithm'] = df['algorithm'].str.replace("strategic_threshold", "Threshold")
df['algorithm'] = df['algorithm'].str.replace("strategicTrue", "Parallel")
df['algorithm'] = df['algorithm'].str.replace("strategicFalse", "Iterative")

In [10]:
utils.latexify(5,5, font_scale=3)
temp_df = df
temp_df.sort_values(["algorithm"], inplace=True)
hue_order=["Non-Strategic", "Threshold", "Bruteforce", "Iterative"]
# hue_order=["Non-Strategic", "Threshold", "Bruteforce", "Iterative", "Parallel"]
linestyles = [linestyles_dict[algo] for algo in hue_order]
markers = [markers_dict[algo] for algo in hue_order]
g = sns.catplot(x="m", y="utility", hue="algorithm", data=temp_df, kind="point", hue_order=hue_order,
            aspect=1.6,legend=False,palette=col_dict ,linestyles=linestyles,
           markers=markers,ci=40)
plt.setp(g.ax.lines,linewidth=2)
axes = plt.gca()
y_max = axes.get_ylim()[1]
plt.vlines(5.5,0.02,y_max-0.005,linestyle="--")
plt.legend(shadow=None, loc=4,labelspacing=0.1, handletextpad=0.3)
plt.ylabel("Utility, $u(\\pi, \\gamma)$")
plt.xlabel("$m$")
plt.ylim(ymin=0)
plt.savefig("./figures/optimal/{name}_u_cost_{cost_method}_kappa_{kappa}.pdf".format(name=name, cost_method=cost_method, kappa=str(kappa).replace('.','')),bbox_inches='tight', )

/var/folders/gy/xwt8ngc15hj3w_8vq_bk0yz80000gt/T/ipykernel_93946/4227176900.py:8: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=('ci', 40)` for the same effect.

  g = sns.catplot(x="m", y="utility", hue="algorithm", data=temp_df, kind="point", hue_order=hue_order,
